## Estadística Inferencial 

- Intervalos de confianza:
  - Muestra 
  - Poblacion 
- Pruebas de hipótesis


In [4]:

# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
#import scipy.stats as stats
import scipy.stats as st
import scipy.stats as stats
from scipy.stats import shapiro, poisson, chisquare, expon, kstest


# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

In [5]:
df = pd.read_csv("bank-additional-clean-nonulls.csv", index_col = 0)
df = df[df['age'] > 0]
df['marital'] = df['marital'].replace({'noviembre': 'married'})
df.head(2)

,income,kidhome,teenhome,dt_customer,numwebvisitsmonth,id,job,marital,education,default,housing,loan,contact,duration,campaign,previous,poutcome,empvarrate,conspriceidx,consconfidx,nremployed,y,date,latitude,longitude,contact_month,contact_year,age_cat,age,euribor3m,pdays
0,161770,1,0,2012-04-04,29,089b39d8-e4d0-461b-87d4-814d71e0e079,housemaid,married,basic 4y,No,No,No,telephone,261,1,0,nonexistent,1.1,93.994,-36.4,5191,no,2-agosto-2019,41.495,-71.233,agosto,2019.0,Adultos mayores,40.005436,4.857000,4.4
1,85477,1,1,2012-12-30,7,e9d37224-cb6f-4942-98d7-46672963d097,services,married,high school,noviembre,No,No,telephone,149,1,0,nonexistent,1.1,93.994,-36.4,5191,no,14-septiembre-2016,34.601,-83.923,septiembre,2016.0,Mediana edad,57.000000,3.603174,4.8


## Receta: 

1. Definir el problema
2. Extraccion de la muestra 
3. Calcular la media , error estandar 
4. Elegir el nivel de confianza: porcentaje de confianza con que voy a calcular la intervalo: 90, 95
5. Aplicar la fórmula
6. Resultado. 

In [12]:
## INtervalo de confianza: 

##30 minutos —media
## 90% ---15minutos a 45 minutos
## 70% --- 20 minutos y 40 minutos

media_duration = df['duration'].mean()
media_duration

error = stats.sem(df['duration'])
error

nivel_confianza = 0.9
grados_de_libertad = len(df['duration'])-1

valor_critico = stats.t.ppf((1 + nivel_confianza) / 2, df=grados_de_libertad)

limite_inferior = media_duration-valor_critico*error
limite_superior =  media_duration+valor_critico*error

print(media_duration)
print(limite_inferior)
print(limite_superior)


257.74681865767127
255.69439777549098
259.79923953985156


## Pruebas de hipotesis

1. Plantear la hipotesis
   - Hipotesis Nula: Los grupos son iguales
   - Hipotesis Alternativa: Los grupos no son iguales
2. Seleccionar los grupos
3. Prueba estadistica: seleccionar una prueba para decidir si la hipotesis se cumple o no: t de student, main whitney, .... anova, chi caudrado...
   1. Condiciones a comprobar
      1. Datos normales o no: shapiro, kolmogorov
      2. Homogenidad de varianza: levene, barlett
4. Interpreto de los resultados




In [16]:
## dos grupos
 # Poner las hipotesis:
## H0: no hay diferencias significativas
# H1: hay diferencias significativas

grupo_1 = df[df['education']== 'basic 4y']["income"]
grupo_2 = df[df['education']== 'high school']["income"]


-  Normalidad de los datos: verificar si los datos siguen una distribución normal
   -  Prueba de Shapiro-Wilk:
      - Cuándo usarla:
        - Tamaños de muestra pequeños o medianos (idealmente entre 3 y 50 observaciones).
      - Hipótesis:
        - H0: Los datos siguen una distribución normal.
        - H1: Los datos no siguen una distribución normal.
      - 
   -  Prueba de Kolmogorov-Smirnov
      -  Cuándo usarla
         -  Tamaños de muestra grandes (más de 50 observaciones).
           - Comparar si una muestra sigue una distribución específica (no solo normal). Por ejemplo: comparar si los datos siguen una distribución normal
      -  Hipótesis:
        - H0: Los datos siguen una distribución normal.
        - H1: Los datos no siguen una distribución normal.


In [19]:
## Aplicar test shapiro, kolmogorov sensibles al tamaño de la muestra, grupo grande kol, grupo pequeño shapiro
if len(grupo_1)> 5000:
    p_valor= stats.kstest(grupo_1, 'norm').pvalue
else:
    p_valor = stats.shapiro(grupo_1).pvalue


if p_valor > 0.05: 
    print("Valores normales")
else:
     print("Valores no normales")

Valores no normales


In [20]:
if len(grupo_2)> 5000:
    p_valor= stats.kstest(grupo_2, 'norm').pvalue
else:
    p_valor = stats.shapiro(grupo_2).pvalue


if p_valor > 0.05: 
    print("Valores normales")
else:
     print("Valores no normales")

Valores no normales


-  Homogeneidad de varianza: verificar si las varianzas son similares entre los grupos 
      - H0: Las varianzas de los grupos son iguales (homogeneas).
      - H1: Los datos no siguen la distribución esperada.
    
    -  Prueba de Levene
         -  Datos no normales
         -  Hay outliers
         -  No estás segura de la situación
   -  Prueba de Barlett
         -  los datos son normales

In [21]:
## Mirar la homegeneidad de las varianzas

p_valor_varianza = stats.levene(grupo_1, grupo_2).pvalue
p_valor_varianza

if(p_valor_varianza> 0.05):
    print("Varianzas iguales, cumple hipótesis nula")
else:
    print("Varianzas no son iguales, cumple hipótesis nula")

Varianzas no son iguales


In [22]:
## aplico el test estadistico
t_stat, p_valor=stats.mannwhitneyu(grupo_1, grupo_2)
if(p_valor > 0.05):
    print("Se cumple mi hipotesis nula")
else: 
    print("No se cumple mi hipotesis nula")

Se cumple mi hipotesis nula


In [23]:
def prueba_hipotesis(*args):
    """
    Realiza una prueba de hipótesis para comparar grupos.
    1. Primero verifica si los datos son normales usando el test de Shapiro-Wilk o Kolmogorov-Smirnov.
    2. Si los datos son normales, usa Bartlett para probar igualdad de varianzas. Si no son normales, usa Levene.
    3. Si las varianzas son iguales, usa el t-test de Student; si no, usa la versión de Welch.
    4. Si los datos no son normales, usa el test de Mann-Whitney (alternativa no paramétrica).

    Parámetros:
    *args: listas o arrays con los datos de cada grupo.

    Retorna:
    dict con resultados del test de normalidad, varianza e hipótesis.
    """
    
    # Verificar si hay al menos dos grupos
    if len(args) < 2:
        raise ValueError("Se necesitan al menos dos conjuntos de datos para realizar la prueba.")
    
    # Comprobar normalidad en cada grupo
    normalidad = []
    for grupo in args:
        if len(grupo) > 50:
            p_valor_norm = stats.kstest(grupo, 'norm').pvalue  # Kolmogorov-Smirnov si n > 50
        else:
            p_valor_norm = stats.shapiro(grupo).pvalue  # Shapiro-Wilk si n <= 50
        normalidad.append(p_valor_norm > 0.05)

    datos_normales = all(normalidad)  # True si todos los grupos son normales

    # Prueba de igualdad de varianzas
    if datos_normales:
        p_valor_varianza = stats.bartlett(*args).pvalue  # Test de Bartlett si los datos son normales
    else:
        p_valor_varianza = stats.levene(*args, center="median").pvalue  # Test de Levene si no son normales

    varianzas_iguales = p_valor_varianza > 0.05

    # Aplicar el test adecuado
    if datos_normales:
        if varianzas_iguales:
            t_stat, p_valor = stats.ttest_ind(*args, equal_var=True)
            test_usado = "t-test de Student (varianzas iguales)"
        else:
            t_stat, p_valor = stats.ttest_ind(*args, equal_var=False)
            test_usado = "t-test de Welch (varianzas desiguales)"
    else:
        t_stat, p_valor = stats.mannwhitneyu(*args)
        test_usado = "Mann-Whitney U (prueba no paramétrica)"

    # Nivel de significancia
    alfa = 0.05

    # Resultados
    resultado = {
        "Test de Normalidad": normalidad,
        "Datos Normales": datos_normales,
        "p-valor Varianza": p_valor_varianza,
        "Varianzas Iguales": varianzas_iguales,
        "Test Usado": test_usado,
        "Estadístico": t_stat,
        "p-valor": p_valor,
        "Conclusión": "Rechazamos H0 (Diferencias significativas)" if p_valor < alfa else "No se rechaza H0 (No hay diferencias significativas)"
    }

    # Imprimir resultados de manera más clara
    print("\n📊 **Resultados de la Prueba de Hipótesis** 📊")
    print(f"✅ Test de Normalidad: {'Sí' if datos_normales else 'No'}")
    print(f"   - Normalidad por grupo: {normalidad}")
    print(f"✅ Test de Varianza: {'Iguales' if varianzas_iguales else 'Desiguales'} (p = {p_valor_varianza:.4f})")
    print(f"✅ Test aplicado: {test_usado}")
    print(f"📉 Estadístico: {t_stat:.4f}, p-valor: {p_valor:.4f}")
    print(f"🔍 Conclusión: {resultado['Conclusión']}\n")

    return resultado

In [24]:
prueba_hipotesis(grupo_1, grupo_2)


📊 **Resultados de la Prueba de Hipótesis** 📊
✅ Test de Normalidad: No
   - Normalidad por grupo: [False, False]
✅ Test de Varianza: Desiguales (p = 0.0111)
✅ Test aplicado: Mann-Whitney U (prueba no paramétrica)
📉 Estadístico: 21152249.0000, p-valor: 0.0521
🔍 Conclusión: No se rechaza H0 (No hay diferencias significativas)



{'Test de Normalidad': [False, False],
 'Datos Normales': False,
 'p-valor Varianza': 0.011100694428226266,
 'Varianzas Iguales': False,
 'Test Usado': 'Mann-Whitney U (prueba no paramétrica)',
 'Estadístico': 21152249.0,
 'p-valor': 0.052125869808146034,
 'Conclusión': 'No se rechaza H0 (No hay diferencias significativas)'}

In [25]:
grupo_casadas = df[df["marital"] == "married"]["duration"]
grupo_solteras = df[df["marital"] == "single"]["duration"]
prueba_hipotesis(grupo_casadas, grupo_solteras)


📊 **Resultados de la Prueba de Hipótesis** 📊
✅ Test de Normalidad: No
   - Normalidad por grupo: [False, False]
✅ Test de Varianza: Iguales (p = 0.1008)
✅ Test aplicado: Mann-Whitney U (prueba no paramétrica)
📉 Estadístico: 157034711.0000, p-valor: 0.4687
🔍 Conclusión: No se rechaza H0 (No hay diferencias significativas)



{'Test de Normalidad': [False, False],
 'Datos Normales': False,
 'p-valor Varianza': 0.10084402738150801,
 'Varianzas Iguales': True,
 'Test Usado': 'Mann-Whitney U (prueba no paramétrica)',
 'Estadístico': 157034711.0,
 'p-valor': 0.4686607752659886,
 'Conclusión': 'No se rechaza H0 (No hay diferencias significativas)'}